In [1]:
class Sentiment:
    NEGATIVE = "Negative"
    NEUTRAL = "Neutral"
    POSITIVE = "Positive"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score 
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: 
            return Sentiment.POSITIVE


In [4]:
import json

file_name = "Books_small.json"

reviews = []
with open(file_name) as f: 
    for lines in f:
        review = json.loads(lines)
        reviews.append((Review(review['reviewText'], review['overall'])))

reviews[100].sentiment

'Neutral'

In [6]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size= 0.33, random_state= 42)


In [13]:
print(training[5].sentiment)

Positive


In [16]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training] 

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]



train_x[0]

"Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down."

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(train_x)



<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 41455 stored elements and shape (670, 7372)>

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vector = vectorizer.fit_transform(train_x)

print(train_x[0])
print(train_x_vector[0].toarray())


Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
[[0 0 0 ... 0 0 0]]
